In [2]:
from vnpy.trader.utility import load_json, save_json
# 更新配置文件数据库名称
database_name = "RQDATA.db"

setting = load_json("vt_setting.json")
setting.update({"database.database": database_name})
save_json("vt_setting.json", setting)

In [4]:
from vnpy.trader.engine import MainEngine, EventEngine
from vnpy.app.data_manager.engine import ManagerEngine

manager = ManagerEngine(MainEngine, EventEngine)

在使用天勤量化之前，默认您已经知晓并同意以下免责条款，如果不同意请立即停止使用：https://www.shinnytech.com/blog/disclaimer/


In [8]:
# 将CSV行情数据导入数据库
manager.import_data_from_csv(
        file_path = "rb888_SHFE.csv",
        symbol    = "rb888",
        exchange  = "SHFE",
        interval  = "1m",
        datetime_head = "datetime",
        open_head   = "open",
        high_head   = "high",
        low_head    = "low",
        close_head  = "close",
        volume_head = "volume",
        open_interest_head = "open_interest",
        datetime_format    = "%Y-%m-%d %H:%M:%S"
    )

(datetime.datetime(2010, 1, 4, 9, 0),
 datetime.datetime(2020, 5, 29, 22, 59),
 753000)

In [4]:
from vnpy.trader.constant import Interval, Exchange

# 获取数据库中所有品种的Bar数据统计信息，包括字段：symbol，exchange，interval，count，start，end
bar_data_statistics = manager.get_bar_data_available()

# 从数据接口下载更新数据库中所有品种的Bar数据至最新
for d in bar_data_statistics:
    num = manager.download_bar_data_from_datasource(d["symbol"], Exchange(d["exchange"]), Interval(d["interval"]), d["end"])
    print(f'{d["symbol"]} \t新下载更新Bar数量：{num}')

auth success 
A9999 新下载更新Bar数量：2
AG9999 新下载更新Bar数量：2
AL9999 新下载更新Bar数量：2
AP9999 新下载更新Bar数量：2
AU9999 新下载更新Bar数量：2
B9999 新下载更新Bar数量：2
BB9999 新下载更新Bar数量：2
BU9999 新下载更新Bar数量：2
C9999 新下载更新Bar数量：2
CF9999 新下载更新Bar数量：2
CJ9999 新下载更新Bar数量：2
CS9999 新下载更新Bar数量：2
CU9999 新下载更新Bar数量：2
EB9999 新下载更新Bar数量：2
EG9999 新下载更新Bar数量：2
FB9999 新下载更新Bar数量：2
FG9999 新下载更新Bar数量：2
FU9999 新下载更新Bar数量：2
HC9999 新下载更新Bar数量：2
I9999 新下载更新Bar数量：2
IC8888 新下载更新Bar数量：2
IC9999 新下载更新Bar数量：2
IF8888 新下载更新Bar数量：2
IF9999 新下载更新Bar数量：2
IH8888 新下载更新Bar数量：2
IH9999 新下载更新Bar数量：2
J9999 新下载更新Bar数量：2
JD9999 新下载更新Bar数量：2
JM9999 新下载更新Bar数量：2
L9999 新下载更新Bar数量：2
M9999 新下载更新Bar数量：2
MA9999 新下载更新Bar数量：2
NI9999 新下载更新Bar数量：2
NR9999 新下载更新Bar数量：2
OI9999 新下载更新Bar数量：2
P9999 新下载更新Bar数量：2
PB9999 新下载更新Bar数量：2
PG9999 新下载更新Bar数量：2
PP9999 新下载更新Bar数量：2
RB9999 新下载更新Bar数量：2
RM9999 新下载更新Bar数量：2
RU9999 新下载更新Bar数量：2
SA9999 新下载更新Bar数量：2
SC9999 新下载更新Bar数量：2
SF9999 新下载更新Bar数量：2
SM9999 新下载更新Bar数量：2
SP9999 新下载更新Bar数量：2
SR9999 新下载更新Bar数量：2
SS9999 新下载更新Bar数量：2
T9999 新下载更新Bar

In [7]:
# 保存数据库中的所有品种的 vt_symbol 至当前工作目录
with open("vt_symbol.txt", "w") as f:
    for d in bar_data_statistics:
        vt_symbol = d["symbol"] + "." + d["exchange"]
        f.write(f"{vt_symbol}\n")